In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [3]:
database = 'sqlite_db_pythonsqlite.db'
connection = sql.connect(database)

In [4]:
query = 'SELECT * FROM Facilities'

In [6]:
df = pd.read_sql_query(query, connection)
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [15]:
query1 = '''SELECT s.name, SUM(s.revenue) AS revenue
FROM (
SELECT b.facid, b.memid, f.name, f.guestcost, f.membercost, COUNT( b.facid ) AS facid_count,
CASE
WHEN b.memid = 0
THEN COUNT(b.facid) * f.guestcost
ELSE COUNT(b.facid) * f.membercost
END AS 'revenue'
FROM Bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid, b.memid
) AS s
GROUP BY s.facid
HAVING revenue <= 1000 ;'''

In [16]:
df1 = pd.read_sql_query(query1, connection)
df1

,name,revenue
0,Badminton Court,604.5
1,Table Tennis,90.0
2,Snooker Table,115.0
3,Pool Table,265.0


In [17]:
query2 = '''SELECT m1.surname, m1.firstname, m2.surname AS rec_surname, m2.firstname AS recr_firstname
FROM Members AS m1
LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
WHERE m1.recommendedby IS NOT NULL
ORDER BY m1.surname, m1.firstname ;'''

In [18]:
df2 = pd.read_sql_query(query2, connection)
df2

,surname,firstname,rec_surname,recr_firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Farrell,David,None,None
9,Farrell,Jemima,None,None


In [19]:
query3 = '''SELECT f.name, COUNT(sub.memid) AS facility_member_usage
FROM
	(SELECT facid, memid
     FROM Bookings
     WHERE memid != 0) AS sub
LEFT JOIN Facilities AS f ON sub.facid = f.facid 
GROUP BY sub.facid ;'''

In [20]:
df3 = pd.read_sql_query(query3, connection)
df3

,name,facility_member_usage
0,Tennis Court 1,308
1,Tennis Court 2,276
2,Badminton Court,344
3,Table Tennis,385
4,Massage Room 1,421
5,Massage Room 2,27
6,Squash Court,195
7,Snooker Table,421
8,Pool Table,783


In [23]:
query4 = '''SELECT sub.month, COUNT(sub.memid) AS facility_member_monthly_usage
FROM 
	(SELECT strftime('%m', starttime) as month, memid
	FROM Bookings
	WHERE memid != 0) AS sub
GROUP BY sub.month ;'''

In [24]:
df4 = pd.read_sql_query(query4, connection)
df4

,month,facility_member_monthly_usage
0,07,480
1,08,1168
2,09,1512
